In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.core.display import display, HTML
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import xgboost
import shap
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.inspection import plot_partial_dependence
from sklearn import preprocessing

## 1.2 Students 

In [2]:
#read file, drop dupes that occur across all columns, and then drop the 54 duplicates that are dupes at the ID level (22595-> 22541)
stu = pd.read_csv('files/1.2 Students.csv',parse_dates=['HighSchoolGraduationDate','BirthDate'])
stu.drop_duplicates(inplace=True)
stu = stu[stu.duplicated('nswersguid')==0]

#create grouped language indicator field (english / spanish / other)  - other values aren't frequent
stu['lang'] = np.where((stu['HomeLanguageCode']!= 'English') & (stu['HomeLanguageCode']!= 'Spanish'),
         'Other languages',
         stu['HomeLanguageCode'])

#Drop missing data columns (almost or all missing)
stu.drop(['Race2','Race3','Race4','Race5','ImmigrantIndicator'],axis=1,inplace=True)


# create hs grad indicator, and on-time indicator
stu['HS_Grad'] = np.where(stu['HighSchoolGraduationDate'].isnull(),0,1)
stu['HS_Grad_ontime']  =  np.where(stu['HighSchoolGraduationDate']<'2012',1,0)

#create target var
target1=stu['HS_Grad']
target2=stu['HS_Grad_ontime']

#could create stu[age] from bdate -> 2011
stu['age']  =(pd.to_datetime('2011') - stu['BirthDate']) / np.timedelta64(1, 'Y')

#get part1 of district number
stu['DistrictNumber1'] = stu['DistrictNumber'].str.split('-').str[0]


#drop vars not useful or only used in transform above
stu.drop(['HighSchoolGraduationDate','GradeLevel','HomeLanguageCode',
          'School_Year','SchoolName','BirthDate','DistrictNumber'],
         axis=1,inplace=True)

#get dummies

#first set of features from 1.2 
features = stu.drop(['HS_Grad','HS_Grad_ontime','nswersguid'],axis=1)
features[['Gender','Race1','lang','DistrictNumber1','SchoolNumber']] = features[features.columns[features.dtypes == 'O']].astype('category')

#featuresdum = pd.get_dummies(stu.drop(['HS_Grad','HS_Grad_ontime','nswersguid','DistrictNumber1'],axis=1))
#featuresdum_dropfirst = pd.get_dummies(stu.drop(['HS_Grad','HS_Grad_ontime','nswersguid'],axis=1),drop_first=True)

stu.shape

(22541, 15)

## 1.4 Grades

In [3]:
### 1.4 Grades
grades = pd.read_csv('files/1.4 Student Grades.csv')

# dropping dist/school number - info captured in 1.2
#dropping school year/grade level - info low variance, and possibly represents data leakage for prediction
#dropping dual credit - only 16 'yes' values
grades.drop(['DistrictNumber','SchoolNumber','DualCreditIndicator','School Year','GradeLevel'],axis=1,inplace=True)
grades['AlphaGrade'] = grades['AlphaGrade'].fillna('NA')
grades.shape

(274601, 6)

In [4]:
#408 unique course codes (state course code)
#50k unique section codes
#7133 unique evaluators
#17 unique term types
grades.isnull().sum() / len(grades)

nswersguid           0.000000
Evaluator1StaffId    0.162337
AlphaGrade           0.000000
Term                 0.008092
CourseCode           0.008092
SectionCode          0.008092
dtype: float64

In [5]:
#get unique counts by student
unique = grades.groupby('nswersguid').nunique().reset_index().rename(columns={'Evaluator1StaffId':'UniqueTeachers',
                                                                    'CourseCode':'UniqueCourseCodes',
                                                                    'SectionCode':'UniqueSectionCodes',
                                                                    'AlphaGrade':'UniqueAlphaGrades',
                                                                    'Term':'UniqueTerms'})

#pivot grades for individual grade counts per student
alphas= pd.pivot_table(grades[['nswersguid','AlphaGrade','CourseCode']],
               index=['nswersguid'],columns=['AlphaGrade'],aggfunc='count',dropna=False,fill_value=0).reset_index(col_level=1)
alphas.columns = alphas.columns.droplevel()
alphas.drop(['nswersguid'],axis=1,inplace=True)

#total course count, could be useful to show retakes
totalcourses = grades.groupby('nswersguid').count().reset_index().rename(columns={
                                                                    'SectionCode':'TotalSectionCodes'
                                                                })[['TotalSectionCodes']]
#concatenate the 3 sets of grades features on the index
courses = pd.concat([totalcourses,unique,alphas],axis=1)
courses['retakeCount']= courses['TotalSectionCodes'] - courses['UniqueSectionCodes']
courses = courses.add_prefix('grd_')

#max / min grades
# relative grades to district average?

In [6]:
# add courses to original student file
stu = stu.merge(courses,left_on='nswersguid',right_on='grd_nswersguid').drop('grd_nswersguid',axis=1)

## 1.5 NESA State Assessment

In [7]:
### 1.5 NESA State Assessment
nesa = pd.read_csv('files/1.5 NeSA State Assessment.csv')

#drop redundant columns, and drop rows where no NESA Assessment (after adding missing indicator)
nesa.drop(['DistrictNumber','SchoolNumber','School Year','GradeLevel','Subject'],axis=1,inplace=True)
nesa = nesa.add_prefix('nesa_')
nesa.shape

(22541, 3)

In [8]:
stu = stu.merge(nesa,left_on='nswersguid',right_on='nesa_nswersguid').drop('nesa_nswersguid',axis=1)

In [9]:
#these need to be after train/test split, otherwise probably biasing the results of prediction
#     along with any other calculated fields that ARE NOT calc'd at student level

#add difference between district avg scalescore and student scale score
stu['DistrictStudent_ScaleScore_Diff'] = stu.groupby('DistrictNumber1')['nesa_ScaleScore'].transform('mean') - stu['nesa_ScaleScore']

#add difference between school avg scalescore and student scale score
stu['SchoolStudent_ScaleScore_Diff'] = stu.groupby('SchoolNumber')['nesa_ScaleScore'].transform('mean') - stu['nesa_ScaleScore']

#courses differences between student / district / school
#grd_A, grd_B, etc as % of total course grades earned

In [10]:
#Export HIGH SCHOOL level data -- for use in predictions of College Going Rate + HS Grad Rate + HS Ontime rate
stu.to_csv('Students_HS_v1.csv',index=False)